# 02 - Converters for Quadratic Programs

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1

In [1]:
# %pip install qiskit==1.4.1
# %pip install qiskit-optimization
# %pip install cplex # Installs the Complex Optimizer for qiskit_optimization

#### Restart the kernel after installing any of the missing packages

### Introduction

We can represent optimization problems using Qiskit's optimization module. This module contains the `QuadraticProgram` class, which is a generic and powerful representation for optimization problems. In general, the optimization algorithms are defined for a certain formulation of a quadratic program, so we need to convert our problem to the right type. 

Within the Qiskit optimization module, there exists several methods of converting these quadratic problems. To map a problem to the correct input format, the optimization module contains the following converters.

- `InequalityToEquality`: Convert inequality constraints into equality constraints with additional slack variables.
- `IntegerToBinary`: Convert integer variables into binary variables and corresponding coefficients.
- `LinearEqualityToPenalty`: Convert equality constraints into additional terms of the objective function.
- `LinearInequalityToPenalty`: Convert inequality constraints into additional terms of the objective function.
- `MaximizeToMinimize`: Convert to the equivalent minimization problem.
- `MinimizeToMaximize`: Convert to the equivalent maximization problem.
- `QuadraticProgramToQubo`: A wrapper that includes `InequalityToEquality`, `IntegerToBinary`, `LinearEqualityToPenalty`, `LinearInequalityToPenalty`, and `MaximizeToMinimize` for convenience.

We will use this notebook to cover the conversion methods of `InequalityToEquality`, `IntegerToBinary`, and `LinearEqualityToPenalty`. 

In [2]:
# This code is from:
# https://qiskit-community.github.io/qiskit-optimization/tutorials/02_converters_for_quadratic_programs.html

### InequalityToEquality

We will start with the conversion from inequality to equality. This will use the `InequalityToEquality` converter which, as the name implies, converts inequality constraints into equality constraints with additional slack variables to remove inequality constraints from a `QuadraticProgram`. The upper bounds and the lower bounds of slack variables will be calculated from the difference between the left and right sides of constraints. Signs of slack variables depends on symbols in constraints such as $\leq$ and $\geq$.

The following is an example of a maximization problem with two inequality constraints. Variable $x$ and $y$ are binary variables and variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 3x + 2y + z\\
   & \text{subject to the constraints:}
       & x+y+z \leq 5.5\\
       & & x+y+z \geq 2.5\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem can be seen in the following cells.

In [3]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators.docplex_mp import to_docplex_mp

In [4]:
qp = QuadraticProgram()
qp.binary_var("x")
qp.binary_var("y")
qp.integer_var(lowerbound=0, upperbound=7, name="z")

qp.maximize(linear={"x": 3, "y": 2, "z": 1})
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="LE", rhs=5.5, name="xyz_leq")
qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="GE", rhs=2.5, name="xyz_geq")
print(qp.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + y + z <= 5.5  'xyz_leq'
    x + y + z >= 2.5  'xyz_geq'

  Integer variables (1)
    0 <= z <= 7

  Binary variables (2)
    x y



Import `InequalityToEquality` from `qiskit_optimization.converters`. Create an instance of `InequalityToEquality` and call the `convert` method, passing in the `QuadraticProgram` instance (`qp`) created above.

In [5]:
from qiskit_optimization.converters import InequalityToEquality

In [6]:
ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp)
print(qp_eq.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack + y + z == 5  'xyz_leq'
    x - xyz_geq@int_slack + y + z == 3  'xyz_geq'

  Integer variables (3)
    0 <= z <= 7
    0 <= xyz_leq@int_slack <= 5
    0 <= xyz_geq@int_slack <= 6

  Binary variables (2)
    x y



Note the difference in the constraints after conversion. The inequality constraints have been replaced with equality constraints and have additional integer slack variables, $xyz\_leq@int\_slack$ and $xyz\_geq@int\_slack$. 

The conversion works in the following way. Prior to conversion, the first linear constraint is $x + y + z \leq 5.5$ and there is only one integer variable constraint $0 \leq z \leq 7$. After the conversion, our first linear constraint is changed to $x + xyz\_leq@int\_slack + y + z == 5$ and two additional integer variable constraints of $0 \leq xyz\_leq@int\_slack \leq 5$ and $0 \leq xyz\_geq@int\_slack \leq 6$ to restrict the additional terms $xyz\_leq@int\_slack$ and $xyz\_geq@int\_slack$.

The lower bound of $0$ of the integer variable constraint is for the case where $x$, $y$, and $z$ are all integer values that add up to meet the first linear constraint. If this is the case, then $xyz\_leq@int\_slack$ must be $0$. The upper bound of $0 \leq xyz\_leq@int\_slack \leq 5$ is for the case where $x = 0$, $y = 0$, and $z = 0$. If this is the case, then $xyz\_leq@int\_slack$ must be 5 in order to satisfy the linear constraint. Since all the variables involved can only take on integer values, we drop the decimal from the original constraint. This same approach is applied to the second linear constraint except since this constraint has a $\geq$ symbol, we include a minus before the $xyz\_geq@int\_slack$ to be able to satisfy the case for when $x = 1$, $y = 1$, and $z = 7$. 

\begin{aligned}
   & \text{maximize}
       & 3x + 2y + z\\
   & \text{subject to:}
       & x+y+z+ xyz\_leg\text{@}int\_slack= 5\\
       & & x+y+z+xyz\_geq\text{@}int\_slack= 3\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
       & & xyz\_leg\text{@}int\_slack \in \{0,1,2,3,4,5\} \\
       & & xyz\_geq\text{@}int\_slack \in \{0,1,2,3,4,5,6\} \\
\end{aligned}

We can see how the `interpret` method works next. This method converts the solution of the converted problem back to that of the original problem. To use this method, we first must solve the problem. For the purpose of this notebook, we will use `docplex` to solve. So we will first translate the quadratic problem into a `docplex.mp` model.

In [7]:
from qiskit_optimization.algorithms import CplexOptimizer

cplex_optimizer = CplexOptimizer()

In [8]:
result_orig = cplex_optimizer.solve(qp)
print(result_orig)

fval=8.0, x=1.0, y=1.0, z=3.0, status=SUCCESS


In [14]:
result_eq = cplex_optimizer.solve(qp_eq)
print(result_eq)

fval=8.0, x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0, status=SUCCESS


The `result_eq` of `qp_eq` has 5 variable values (`x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0`) while `result_orig` of the original `qp` has three variable values (`x=1.0, y=1.0, z=3.0`). We can call `InequalityToEquality.interpret` method by passing a list or an array to the method that has values of `qp_eq.variables`. `result_eq.x` has the list of values that each variable takes in the solution in correspondence to their position in the variable list `qp_eq.variables`.

In [15]:
print("Interpreting values of result_eq:", ineq2eq.interpret(result_eq.x))
print("Values of result_orig:", result_orig.x)

Interpreting values of result_eq: [1. 1. 3.]
Values of result_orig: [1. 1. 3.]


Notice that [1., 1., 3.] are the values taken in the solution of the converted problem for the common variables between the converted and original problems (variables: $x$, $y$, $z$). The interpret method shows the same values are the solution of the original problem. This is because the objective function for the converted and original problems is exactly the same. The slack variables are just ensuring equality in the constraints of the converted problem, where the constraints are also exactly same between the original and converted problems, except that the original problem has inequality constraints.

### IntegerToBinary

`IntegerToBinary` converts integer variables into binary variables and coefficients to remove integer variables from `QuadraticProgram`. For converting, the bounded-coefficient encoding proposed in [Practical Integer-to-Binary Mapping for Quantum Annealers](https://arxiv.org/abs/1706.01945) (Eq. (5)) is used. For more detail of the encoding method, feel free to check out the paper.

We will use the output of `InequalityToEquality` as a starting point. Variables $x$ and $y$ are binary variables, while the variable $z$ and the slack variables $xyz\_leq@int\_slack$ and $xyz\_geq@int\_slack$ are integer variables. 

In [29]:
# Print the InequalityToEquality problem for reference

print(qp_eq.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack + y + z == 5  'xyz_leq'
    x - xyz_geq@int_slack + y + z == 3  'xyz_geq'

  Integer variables (3)
    0 <= z <= 7
    0 <= xyz_leq@int_slack <= 5
    0 <= xyz_geq@int_slack <= 6

  Binary variables (2)
    x y



Similar to before, we can call the `convert` method of `IntegerToBinary` to convert the problem.

In [19]:
from qiskit_optimization.converters import IntegerToBinary

In [20]:
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z@0 + 2*z@1 + 4*z@2

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack@0 + 2*xyz_leq@int_slack@1 + 2*xyz_leq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 5  'xyz_leq'
    x - xyz_geq@int_slack@0 - 2*xyz_geq@int_slack@1 - 3*xyz_geq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 3  'xyz_geq'

  Binary variables (11)
    x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
    xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2



We can see that after the conversion we once again have some new variables, this time both in our constraints and in our objective function. The integer variable $z$ is replaced with three binary variables $z@0$, $z@1$, and $z@2$ with coefficients $1$, $2$, and $4$ respectively. The slack variables $xyz\_leq@int\_slack$ and $xyz\_geq@int\_slack$ that were introduced by `InequalityToEquality` are also both replaced with three binary variables with coefficients $1$, $2$, $2$, and $1$, $2$, $3$, respectively. These were done as a result of the methods presented in the paper mentioned above. 

> **Note**: The coefficients mean that the sum of these binary variables with coefficients can be the sum of a subset of $\{1, 2, 4\}$, $\{1, 2, 2\}$, and $\{1, 2, 3\}$ to represent the acceptable values $\{0, ..., 7\}$, $\{0, ..., 5\}$, and $\{0, ..., 6\}$, which respects the lower bound and the upper bound of the original integer variables correctly.

`IntegerToBinary` also provides an `interpret` method that is the functionality for translating a given binary result back to the original integer representation. 

In [21]:
result_eq = cplex_optimizer.solve(qp_eq)
print(result_eq)

fval=8.0, x=1.0, y=1.0, z=3.0, xyz_leq@int_slack=0.0, xyz_geq@int_slack=2.0, status=SUCCESS


In [22]:
result_eq_bin = cplex_optimizer.solve(qp_eq_bin)
print(result_eq_bin)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


`result_eq_bin` has more binary variables due to the conversion by `IntegerToBinary.convert`. `IntegerToBinary.interpret` translates them back to the integer values by aggregating binary variable values associated with the original integer variables of `qp_eq`.

In [24]:
print("interpreting values of result_eq_bin:", int2bin.interpret(result_eq_bin.x))
print("values of result_eq:", result_eq.x)

interpreting values of result_eq_bin: [1. 1. 3. 0. 2.]
values of result_eq: [1. 1. 3. 0. 2.]


### LinearEqualityToPenalty

`LinearEqualityToPenalty` converts linear equality constraints into additional quadratic penalty terms of the objective function to map `QuadraticProgram` to an unconstrained form. An input to the converter has to be a `QuadraticProgram` with only linear equality constraints. Those equality constraints, e.g. $\sum_i a_i x_i  = b$ where $a_i$ and $b$ are numbers and $x_i$ is a variable, will be added to the objective function in the form of $M(b - \sum_i a_i x_i)^2$ where $M$ is a large number as penalty factor.

By default, $M=1e5$. The sign of the term depends on whether the problem type is a maximization or minimization.

We use the output of `IntegerToBinary` as a starting point, where all variables are binary variables and all inequality constraints have been mapped to equality constraints.

In [28]:
# Print the IntegerToBinary problem for reference

print(qp_eq_bin.prettyprint())

Problem name: 

Maximize
  3*x + 2*y + z@0 + 2*z@1 + 4*z@2

Subject to
  Linear constraints (2)
    x + xyz_leq@int_slack@0 + 2*xyz_leq@int_slack@1 + 2*xyz_leq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 5  'xyz_leq'
    x - xyz_geq@int_slack@0 - 2*xyz_geq@int_slack@1 - 3*xyz_geq@int_slack@2 + y
    + z@0 + 2*z@1 + 4*z@2 == 3  'xyz_geq'

  Binary variables (11)
    x y z@0 z@1 z@2 xyz_leq@int_slack@0 xyz_leq@int_slack@1 xyz_leq@int_slack@2
    xyz_geq@int_slack@0 xyz_geq@int_slack@1 xyz_geq@int_slack@2



Once again, we call `convert` from `LinearEqualityToPenalty` to convert the problem.

In [30]:
from qiskit_optimization.converters import LinearEqualityToPenalty

In [31]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.prettyprint())

Problem name: 

Maximize
  -26*x^2 + 26*x*xyz_geq@int_slack@0 + 52*x*xyz_geq@int_slack@1
  + 78*x*xyz_geq@int_slack@2 - 26*x*xyz_leq@int_slack@0
  - 52*x*xyz_leq@int_slack@1 - 52*x*xyz_leq@int_slack@2 - 52*x*y - 52*x*z@0
  - 104*x*z@1 - 208*x*z@2 - 13*xyz_geq@int_slack@0^2
  - 52*xyz_geq@int_slack@0*xyz_geq@int_slack@1
  - 78*xyz_geq@int_slack@0*xyz_geq@int_slack@2 - 52*xyz_geq@int_slack@1^2
  - 156*xyz_geq@int_slack@1*xyz_geq@int_slack@2 - 117*xyz_geq@int_slack@2^2
  - 13*xyz_leq@int_slack@0^2 - 52*xyz_leq@int_slack@0*xyz_leq@int_slack@1
  - 52*xyz_leq@int_slack@0*xyz_leq@int_slack@2 - 52*xyz_leq@int_slack@1^2
  - 104*xyz_leq@int_slack@1*xyz_leq@int_slack@2 - 52*xyz_leq@int_slack@2^2
  + 26*y*xyz_geq@int_slack@0 + 52*y*xyz_geq@int_slack@1
  + 78*y*xyz_geq@int_slack@2 - 26*y*xyz_leq@int_slack@0
  - 52*y*xyz_leq@int_slack@1 - 52*y*xyz_leq@int_slack@2 - 26*y^2 - 52*y*z@0
  - 104*y*z@1 - 208*y*z@2 + 26*z@0*xyz_geq@int_slack@0
  + 52*z@0*xyz_geq@int_slack@1 + 78*z@0*xyz_geq@int_slack@2
  -

After converting, the equality constraints are added to the objective function as additional terms with the default penalty factor provided by Qiskit optimization. The resulting problem is now  QUBO and compatible with many quantum optimization algorithms such as VQE, QAOA, and so on.

This gives the same result as before. Let us see how `interpret` works for this case.

In [32]:
result_eq_bin = cplex_optimizer.solve(qp_eq_bin)
print(result_eq_bin)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


In [33]:
result_qubo = cplex_optimizer.solve(qubo)
print(result_qubo)

fval=8.0, x=1.0, y=1.0, z@0=1.0, z@1=1.0, z@2=0.0, xyz_leq@int_slack@0=0.0, xyz_leq@int_slack@1=0.0, xyz_leq@int_slack@2=0.0, xyz_geq@int_slack@0=0.0, xyz_geq@int_slack@1=1.0, xyz_geq@int_slack@2=0.0, status=SUCCESS


We can see that the result of the `interpret` method implies that both the original and converted problems have exactly the same solution. This is expected because the converted problem has exactly the same variables as the original problem, the objective has been modified in such a way that we do not have the constraints anymore in the converted problem.

Finally, let us see how we interpret the result of QUBO back to the solution of the original problem `qp`. The cell below shows that the interpreted values are equivalent to the result of the original problem `qp`.

In [36]:
print("interpreting values of result_eq_bin:", lineq2penalty.interpret(result_qubo.x))
print("values of result_eq_bin:", result_eq_bin.x)

interpreting values of result_eq_bin: [1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
values of result_eq_bin: [1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.]


In [37]:
print("result_orig.x", result_orig.x)

x = result_qubo.x
for conv in [lineq2penalty, int2bin, ineq2eq]:
    x = conv.interpret(x)
print("interpreting result_qubo.x", x)

result_orig.x [1. 1. 3.]
interpreting result_qubo.x [1. 1. 3.]


### Conclusion

Congratulations! You have successfully practiced conversions of a `QuadraticProgram` instance across multiple forms to represent the original problem. While we only explored three conversions in this notebook, as mentioned at the start of the notebook, there are multiple conversions available. Each conversion has their own purpose and it depends on what final form you want your `QuadraticProgram` to take!